Aditi Surma: 84186501
\
Cathy Lei: 12532537
\
Lilian Wang: 35169481

# **Heart Disease Data Analysis**

**Introduction:**
\
\
&emsp; &emsp;Cardiovascular disease (CVD) is a leading cause of death globally, and deaths due to CVD have only increased (He et al. 80). To illustrate, in 1990 there were 12.1 million global CVD-caused deaths, which increased to 18.6 million deaths in 2019 (He et al. 80). CVD includes any condition which may impact the cardiovascular system, such as coronary heart disease or heart failure. Since CVD harms numerous patients globally, it is important to develop a system which can diagnose CVD as early as possible to ensure immediate treatment and possible recovery. This leads to our project’s central question: **Based on a patient’s resting systolic blood pressure, serum cholesterol levels and maximum achieved heart rate, do they have cardiovascular disease (CVD)?** 

&emsp; &emsp;The purpose of our project is to create a classifying system which uses three predictors, resting systolic blood pressure, serum cholesterol levels and maximum achieved heart rate, to determine whether a patient has CVD. The classifier will be made using a heart disease dataset from UC Irvine’s Machine Learning Repository with data collected by the Cleveland Clinic Foundation. The dataset has 302 data points and a total of 14 attributes: 4 continuous, 9 discrete, and 1 predicted attribute specifying whether an individual has heart disease. As mentioned, 3 continuous variables have been chosen as predictors for our classifier. 


**Preliminary Exploratory Data Analysis:**

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
hd_original_data = pd.read_csv(url, header=None)
hd_original_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


To clean and wrangle this dataset into a tidy format, column names are added in the order given by the original website. The last column, initially referred to as "num", has been renamed to "heart disease presence", and refers to the heart disease status of the patient. This status ranges from 0-4, 0 indicating no presence of heart disease. The original column "thalach" has also been renamed to "max heart rate", and refers to the maximum heart rate achieved. All missing values have been dropped.

In [3]:
hd_original_data.columns = ["age", "sex", "cp", "trestbps(systolic)", "chol", "fbs", "restecg", "max_heart_rate", "exang", "oldpeak", "slope", "ca", "thal", "heart_disease_presence"]
hd_original_data['heart_disease_presence'] = pd.Categorical(hd_original_data.heart_disease_presence)
hd_data = hd_original_data[(hd_original_data['age'] != "?")
                           & (hd_original_data['sex'] != "?")
                           & (hd_original_data['trestbps(systolic)'] != "?")
                           & (hd_original_data['chol'] != "?")
                           & (hd_original_data['fbs'] != "?")
                           & (hd_original_data['restecg'] != "?")
                           & (hd_original_data['max_heart_rate'] != "?")
                           & (hd_original_data['exang'] != "?")
                           & (hd_original_data['oldpeak'] != "?")
                           & (hd_original_data['slope'] != "?")
                           & (hd_original_data['ca'] != "?")
                           & (hd_original_data['thal'] != "?")
                           & (hd_original_data['heart_disease_presence'] != "?")]
hd_data

,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3


For our project, we will split 75% of the data to use as our training data and the other 25% as our testing data.

In [4]:
hd_train, hd_test = train_test_split(hd_data, test_size=0.25, random_state=123, stratify=hd_data["heart_disease_presence"]) 
hd_train

,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
295,41.0,1.0,2.0,120.0,157.0,0.0,0.0,182.0,0.0,0.0,1.0,0.0,3.0,0
100,45.0,1.0,4.0,115.0,260.0,0.0,2.0,185.0,0.0,0.0,1.0,0.0,3.0,0
279,58.0,0.0,4.0,130.0,197.0,0.0,0.0,131.0,0.0,0.6,2.0,0.0,3.0,0
163,58.0,0.0,4.0,100.0,248.0,0.0,2.0,122.0,0.0,1.0,2.0,0.0,3.0,0
38,55.0,1.0,4.0,132.0,353.0,0.0,0.0,132.0,1.0,1.2,2.0,1.0,7.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,58.0,1.0,3.0,140.0,211.0,1.0,2.0,165.0,0.0,0.0,1.0,0.0,3.0,0
34,44.0,1.0,3.0,130.0,233.0,0.0,0.0,179.0,1.0,0.4,1.0,0.0,3.0,0
98,52.0,1.0,2.0,134.0,201.0,0.0,0.0,158.0,0.0,0.8,1.0,1.0,3.0,0
19,49.0,1.0,2.0,130.0,266.0,0.0,0.0,171.0,0.0,0.6,1.0,0.0,3.0,0


To explore our dataset, we found the count and percentage of each level of heart disease presence:

In [5]:
explore_hd_grouped = (hd_train.groupby('heart_disease_presence').count())
explore_hd = explore_hd_grouped[["age"]].rename(columns={"age":"count"})
explore_hd = explore_hd.assign(
    percentage=100*explore_hd['count']/len(hd_train)
)
explore_hd

,count,percentage
heart_disease_presence,,
0,120,54.054054
1,40,18.018018
2,26,11.711712
3,26,11.711712
4,10,4.504505


The means of each predicator for the individual heart disease presence:

In [6]:
hd_predictors = hd_train[["trestbps(systolic)", "chol", "max_heart_rate", "heart_disease_presence"]]
hd_predictors.columns = ["trestbps(systolic) mean", "chol mean", "max_heart_rate mean", "heart_disease_presence"]
hd_0 = hd_predictors[hd_predictors["heart_disease_presence"] == 0]
hd_0 = pd.DataFrame(hd_0.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_0

hd_1 = hd_predictors[hd_predictors["heart_disease_presence"] == 1]
hd_1 = pd.DataFrame(hd_1.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_1.index=['1']

hd_2 = hd_predictors[hd_predictors["heart_disease_presence"] == 2]
hd_2 = pd.DataFrame(hd_2.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_2.index=['2']

hd_3 = hd_predictors[hd_predictors["heart_disease_presence"] == 3]
hd_3 = pd.DataFrame(hd_3.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_3.index=['3']

hd_4 = hd_predictors[hd_predictors["heart_disease_presence"] == 4]
hd_4 = pd.DataFrame(hd_4.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_4.index=['4']

hd_all = [hd_0, hd_1, hd_2, hd_3, hd_4]

hd_mean2 = pd.concat(hd_all)
hd_mean2.index.name = "heart_disease_presence"
hd_mean2

,trestbps(systolic) mean,chol mean,max_heart_rate mean
heart_disease_presence,,,
0,129.483333,244.075000,158.875000
1,132.750000,249.325000,145.900000
2,131.653846,259.692308,135.423077
3,133.730769,238.807692,134.115385
4,136.900000,253.100000,139.400000


Graphs representing these means:

In [7]:
hd_mean2_ = hd_mean2.reset_index()
hd_mean2_["heart_disease_presence"] = pd.Categorical(hd_mean2_.heart_disease_presence)
hdp_vs_max_htrt = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("max_heart_rate mean", title="Maximum heart rate (BPM)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_max_htrt

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [8]:
hdp_vs_chol = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("chol mean", title="Serum cholesterol level (mg/dl)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_chol

alt.Chart(...)

In [9]:
hdp_vs_restbps = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("trestbps(systolic) mean", title="Systolic resting blood pressure (mm Hg)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_restbps

alt.Chart(...)

Since our research question focuses more on the general question of whether we can predict a person has heart disease or not, we have a modified table representing 0 as no and 1-4 as yes.

In [10]:
hd_test["heart_disease_presence"] = (hd_test['heart_disease_presence']).astype(str)
hd_general_test = hd_test.replace({"heart_disease_presence": {"0":"no", "1":"yes", "2":"yes", "3":"yes", "4":"yes"}})

In [11]:
hd_train["heart_disease_presence"] = (hd_train['heart_disease_presence']).astype(str)
hd_general_train = hd_train.replace({"heart_disease_presence": {"0":"no", "1":"yes", "2":"yes", "3":"yes", "4":"yes"}})
hd_general_train

,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
295,41.0,1.0,2.0,120.0,157.0,0.0,0.0,182.0,0.0,0.0,1.0,0.0,3.0,no
100,45.0,1.0,4.0,115.0,260.0,0.0,2.0,185.0,0.0,0.0,1.0,0.0,3.0,no
279,58.0,0.0,4.0,130.0,197.0,0.0,0.0,131.0,0.0,0.6,2.0,0.0,3.0,no
163,58.0,0.0,4.0,100.0,248.0,0.0,2.0,122.0,0.0,1.0,2.0,0.0,3.0,no
38,55.0,1.0,4.0,132.0,353.0,0.0,0.0,132.0,1.0,1.2,2.0,1.0,7.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,58.0,1.0,3.0,140.0,211.0,1.0,2.0,165.0,0.0,0.0,1.0,0.0,3.0,no
34,44.0,1.0,3.0,130.0,233.0,0.0,0.0,179.0,1.0,0.4,1.0,0.0,3.0,no
98,52.0,1.0,2.0,134.0,201.0,0.0,0.0,158.0,0.0,0.8,1.0,1.0,3.0,no
19,49.0,1.0,2.0,130.0,266.0,0.0,0.0,171.0,0.0,0.6,1.0,0.0,3.0,no


In [37]:
hd_predictors_2 = hd_general_train[["trestbps(systolic)", "chol", "max_heart_rate", "heart_disease_presence"]]
hd_predictors_2.columns = ["trestbps(systolic) mean", "chol mean", "max_heart_rate mean", "heart_disease_presence"]

hd_no = hd_predictors_2[hd_predictors_2["heart_disease_presence"] == "no"]
hd_no = pd.DataFrame(hd_no.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_no.index=["no"]

hd_yes = hd_predictors_2[hd_predictors_2["heart_disease_presence"] == "yes"]
hd_yes = pd.DataFrame(hd_yes.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_yes.index=["yes"]

hd_general_all = [hd_no, hd_yes]
hd_general_mean2 = pd.concat(hd_general_all)
hd_general_mean2.index.name = "heart_disease_presence"
hd_general_mean2

,trestbps(systolic) mean,chol mean,max_heart_rate mean
heart_disease_presence,,,
no,129.483333,244.075000,158.875000
yes,133.127451,249.656863,139.588235


In [39]:
hd_general_mean2_ = hd_general_mean2.reset_index()
hd_general_mean2_["heart_disease_presence"] = pd.Categorical(hd_general_mean2_.heart_disease_presence)
hdp_vs_max_htrt_general = (
    alt.Chart(hd_general_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("max_heart_rate mean", title="Maximum heart rate (BPM)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_max_htrt_general

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [40]:
hdp_vs_chol_general = (
    alt.Chart(hd_general_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("chol mean", title="Serum cholesterol level (mg/dl)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_chol_general

alt.Chart(...)

In [41]:
hdp_vs_restbps_general = (
    alt.Chart(hd_general_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("trestbps(systolic) mean", title="Systolic resting blood pressure (mm Hg)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_restbps_general

alt.Chart(...)

In [12]:
chol_vs_restbps = (
    alt.Chart(hd_general_train)
    .mark_circle()
    .encode(
        x=alt.X("chol", title="Serum cholesterol level (mg/dl)", scale=alt.Scale(zero=False)),
        y=alt.Y("trestbps(systolic)", title="Systolic resting blood pressure (mm Hg)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
chol_vs_restbps

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [13]:
chol_vs_max_htrt = (
    alt.Chart(hd_general_train)
    .mark_circle()
    .encode(
        x=alt.X("chol", title="Serum cholesterol level (mg/dl)", scale=alt.Scale(zero=False)),
        y=alt.Y("max_heart_rate", title="Maximum heart rate (BPM)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
chol_vs_max_htrt

alt.Chart(...)

In [14]:
restbps_vs_max_htrt = (
    alt.Chart(hd_general_train)
    .mark_circle()
    .encode(
        x=alt.X("trestbps(systolic)", title="Systolic resting blood pressure (mm Hg)", scale=alt.Scale(zero=False)),
        y=alt.Y("max_heart_rate", title="Maximum heart rate (BPM)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
restbps_vs_max_htrt

alt.Chart(...)

**Data Analysis**

To choose our best K for the classifier, we will standardize the data, create a pipeline, and perform cross-validation with GridSearchCV:

In [15]:
hd_preprocessor = make_column_transformer(
    (StandardScaler(), ["trestbps(systolic)", "max_heart_rate", "chol"]),
)

param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 30, 1),
}

hd_tune_pipe = make_pipeline(hd_preprocessor, KNeighborsClassifier())

knn_tune_grid = GridSearchCV(
    hd_tune_pipe, param_grid, cv=4,
)

knn_tune_grid

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['trestbps(systolic)',
                                                                          'max_heart_rate',
                                                                          'chol'])])),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': range(2, 30)})

In [16]:
X_tune = hd_general_train[["trestbps(systolic)", "max_heart_rate", "chol"]]
y_tune = hd_general_train["heart_disease_presence"]

knn_model_grid = knn_tune_grid.fit(X_tune, y_tune)

accuracies_grid = pd.DataFrame(knn_model_grid.cv_results_)
knn_model_grid
accuracies_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026967,0.005409,0.023286,0.000311,2,{'kneighborsclassifier__n_neighbors': 2},0.517857,0.535714,0.600000,0.581818,0.558847,0.033304,28
1,0.018462,0.000915,0.021801,0.000708,3,{'kneighborsclassifier__n_neighbors': 3},0.607143,0.607143,0.654545,0.618182,0.621753,0.019462,18
2,0.017831,0.000069,0.021179,0.000204,4,{'kneighborsclassifier__n_neighbors': 4},0.589286,0.553571,0.654545,0.600000,0.599351,0.036207,27
3,0.017853,0.000098,0.021370,0.000172,5,{'kneighborsclassifier__n_neighbors': 5},0.571429,0.642857,0.654545,0.654545,0.630844,0.034634,13
4,0.010352,0.004608,0.009378,0.000760,6,{'kneighborsclassifier__n_neighbors': 6},0.571429,0.607143,0.654545,0.636364,0.617370,0.031456,20
5,0.007612,0.000098,0.009233,0.000526,7,{'kneighborsclassifier__n_neighbors': 7},0.553571,0.625000,0.618182,0.636364,0.608279,0.032246,23
6,0.007485,0.000487,0.008338,0.000396,8,{'kneighborsclassifier__n_neighbors': 8},0.553571,0.607143,0.636364,0.618182,0.603815,0.030827,25
7,0.006580,0.000349,0.008119,0.000246,9,{'kneighborsclassifier__n_neighbors': 9},0.607143,0.625000,0.654545,0.672727,0.639854,0.025432,4
8,0.007062,0.000067,0.008447,0.000263,10,{'kneighborsclassifier__n_neighbors': 10},0.589286,0.660714,0.672727,0.636364,0.639773,0.031957,5
9,0.006972,0.000080,0.008165,0.000156,11,{'kneighborsclassifier__n_neighbors': 11},0.553571,0.642857,0.654545,0.672727,0.630925,0.045911,12


In [17]:
accuracy_versus_k_grid = (
    alt.Chart(accuracies_grid, title="Grid Search")
    .mark_line(point=True)
    .encode(
        x=alt.X(
            "param_kneighborsclassifier__n_neighbors",
            title="Neighbors",
            scale=alt.Scale(zero=False),
        ),
        y=alt.Y(
            "mean_test_score", 
            title="Mean Test Score", 
            scale=alt.Scale(zero=False)
        ),
    )
    .configure_axis(labelFontSize=10, titleFontSize=15)
    .properties(width=400, height=300)
)

accuracy_versus_k_grid

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

From the graph above, we can see that K=28 gives us the highest accuracy, given K values from 2-14. So, we will use K=13 for our classification.

Then, we will train the classifier and predict the labels:

In [18]:
knn = KNeighborsClassifier(n_neighbors=28) 

X = hd_general_train.loc[:, ["trestbps(systolic)", "max_heart_rate", "chol"]]
y = hd_general_train["heart_disease_presence"]

knn_fit = make_pipeline(hd_preprocessor, knn).fit(X, y)

hd_test_predictions = hd_general_test.assign(
    predicted = knn_fit.predict(hd_test.loc[:, ["trestbps(systolic)", "max_heart_rate", "chol"]])
)
hd_test_predictions[['heart_disease_presence', 'predicted']]

,heart_disease_presence,predicted
183,no,no
204,no,no
151,no,yes
218,no,yes
93,no,no
...,...,...
177,yes,yes
75,no,yes
291,no,no
188,yes,no


Our accuracy from this model is:

In [19]:
hd_acc = knn_fit.score(
    hd_general_test.loc[:, ["trestbps(systolic)", "max_heart_rate", "chol"]],
    hd_general_test["heart_disease_presence"]
)
hd_acc

0.68

In [20]:
pd.crosstab(
    hd_test_predictions["heart_disease_presence"],
    hd_test_predictions["predicted"]
)

predicted,no,yes
heart_disease_presence,,
no,30,10
yes,14,21


**Methods:**

&emsp; &emsp; The variables we chose as predictors are systolic blood pressure, serum cholesterol levels and maximum achieved heart rate. These were chosen based on previous clinical studies investigating their correlation with the presence of CVD. To elaborate, a study conducted at Zhengzhou University found a strong, positive, linear correlation between systolic blood pressure, and the hazard ratio for CVD-caused mortality (He et al. 85). This conveys that systolic blood pressure is a significant indicator of CVD, and therefore an useful predictor for our classifier. In addition, the Framingham Study revealed a strong, positive, linear correlation between serum cholesterol and the instance of coronary heart disease, a branch of CVD (William B. Kannel, et al. 43). This justifies the importance of serum cholesterol levels as a predictor of CVD in our classifier. Finally, a study conducted by the Cardiovascular Institute and Fu Wai Hospital found that a heart rate above 90 beats per minute has the greatest hazard risk for CVD and a heart rate above 75 had the greatest number of patients diagnosed with CVD (Qunxia Mao et al. 1644). Thus, heart rate is a vital indicator for CVD and a predictor in our classifier.  

&emsp; &emsp; We could visualize all three quantitative variables with a scatter plot with two variables on the axes, one variable shown through the points’ color gradient, and presence of CVD through point shape. We could also make multiple scatter plot facets with different pairs of predictor variables. 

**Expected outcomes and significance:**

&emsp; &emsp; Based on previous studies, we expect our classifier to be more likely to predict patients as having CVD if they score highly on any predictor variable. Resting systolic blood pressures above 130 bpm (He et al. 85), heart rates over 90 bpm (Mao 1644), or scoring highly on multiple predictors is expected to be particularly strongly linked to CVD. 

&emsp; &emsp; Our classifier will allow doctors to efficiently assess whether patients have CVD, enabling millions of global CVD patients to more quickly obtain treatment. This will also contribute questions to CVD research. Future projects may investigate changes in these predictors over time. For instance, if we find that systolic blood pressure is a strong CVD predictor, it would be worth asking if one is more likely to have CVD if their systolic blood pressure increased quickly over a few years versus more gradually. Our results will also lead to questions to further narrow predictors. For instance, if cholesterol proves to be a significant predictor, we could explore cholesterol gained from diet versus disease. Overall, our project on CVD offers significant clinical and research value. 

**Citations**
\
\
He, L. et al. “Relationship of resting heart rate and blood pressure with all-cause and cardiovascular  disease mortality.” Public Health, vol. 208, June 2022, pages 80-88, Science Direct, https://www.sciencedirect.com/science/article/pii/S0033350622001032. Accessed March 11, 2023. 
\
\
Kannel, W. B. et al. “Factors of risk in the development of coronary heart disease--six year follow-up experience. The Framingham Study.” Annals of Internal Medicine, vol. 55, no. 1, July 1961, pages 33-50, National Library of Medicine. https://pubmed.ncbi.nlm.nih.gov/13751193/. Accessed March 11, 2023. 
\
\
Mao, Qunxia et al. “Heart rate influence on incidence of cardiovascular disease among adults in China.” International Journal of Epidemiology, vol. 39, no. 6, December 2010, pages 1638–1646, Oxford Academic. https://academic.oup.com/ije/article/39/6/1638/738860?login=false. Accessed March 11, 2023. 